In [1]:
# -*- coding: utf-8 -*-
"""SimplerMambaSSM.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1g9qpeVcFa0ca0cnhmqusO4RZtQdh9umY
"""

#!pip install mamba-ssm causal-conv1d

#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

#!mkdir differentattention

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
from tqdm import tqdm
from mamba_ssm import Mamba
import nltk
import pandas as pd

nltk.download("brown")
from nltk.corpus import brown
import random
from sklearn.model_selection import train_test_split
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# hyperparams
epochs = 1
lr = 1e-3
batch_size = 48
# 2048@48 / 7 chars per word (avg) = 292 words (30 min) 15GB VRAM
block_size = 2048
stride = block_size // 2  # Example stride
# max_iters = 740
# max_iters = 10
print_iters = 100
eval_iters = 10
# eval_interval = 300
n_embed = 384
n_heads = 6
n_layers = 6
dropout = 0.2

# ---------

# train and test splits
# Unique characters - Update to include BOS and EOS tokens
bos_token = "<BOS>"
eos_token = "<EOS>"
chars = sorted(
    list(
        set(
            "".join([" ".join(brown.words(fileid)) for fileid in brown.fileids()])
            + bos_token
            + eos_token
        )
    )
)
print("".join(chars))
vocab_size = len(chars)
print(vocab_size)

# Update the tokenizers to include BOS and EOS
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
stoi[bos_token] = len(chars) - 2  # Assign unique index for BOS
stoi[eos_token] = len(chars) - 1  # Assign unique index for EOS
itos[len(chars) - 2] = bos_token
itos[len(chars) - 1] = eos_token

# Update the encode and decode functions
encode = lambda xx: [stoi[x] for x in xx]
decode = lambda xx: "".join([itos[x] for x in xx])

# Concatenate documents from the Brown Corpus with BOS and EOS tokens
brown_text = "".join(
    [
        bos_token + " ".join(brown.words(fileid)) + eos_token
        for fileid in brown.fileids()
    ]
)

# Encode the Brown Corpus text
data = torch.tensor(encode(brown_text), dtype=torch.long)

# Split into train and validation data
def get_batch(split):
    # generate targets and context
    if split == "train":
        data = train_data
    else:
        data = val_data
    index = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[ind : ind + block_size] for ind in index])
    y = torch.stack([data[ind + 1 : ind + block_size + 1] for ind in index])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "test"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Define your model architecture here

In [2]:
# Continue adding the rest of the model code from above...
# For example, define the class `SelfAttentionHead`, `MultiHeadAttention`, `Block`, and `BigramNeuralNetwork`
# as well as the training loop and saving checkpoint logic.
